## 1. Importation des Bibliothèques

Nous commencerons par importer les bibliothèques nécessaires pour notre projet.

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

## 2. Définition des Chemins vers les Données
Ici, nous définissons les chemins vers les dossiers contenant les images d'entraînement, les masques et les images de test.

In [2]:
# Chemins vers les dossiers du dataset
train_images_path = '/kaggle/input/bccd-dataset-with-mask/BCCD Dataset with mask/train/original'
train_masks_path = '/kaggle/input/bccd-dataset-with-mask/BCCD Dataset with mask/train/mask'
test_images_path = '/kaggle/input/bccd-dataset-with-mask/BCCD Dataset with mask/test/original'
test_masks_path = '/kaggle/input/bccd-dataset-with-mask/BCCD Dataset with mask/test/mask'

# Définir la taille d'image et la taille de lot
IMG_SIZE = (128, 128)
BATCH_SIZE = 16


## 3. Chargement des Données
Nous allons définir une fonction pour charger les images et les masques. Cette fonction va normaliser les images et les masques.

In [ ]:
# Fonction pour charger les images et les masques
def load_dataset(images_path, masks_path):
    images = []
    masks = []
    
    for filename in os.listdir(images_path):
        if filename.endswith('.png'):
            img = load_img(os.path.join(images_path, filename), target_size=IMG_SIZE)
            img = img_to_array(img) / 255.0  # Normalisation
            images.append(img)

            mask = load_img(os.path.join(masks_path, filename), target_size=IMG_SIZE, color_mode='grayscale')
            mask = img_to_array(mask) / 255.0  # Normalisation
            masks.append(mask)
    
    return np.array(images), np.array(masks)

# Charger le dataset d'entraînement
X_train, y_train = load_dataset(train_images_path, train_masks_path)

# Charger le dataset de test
X_test, y_test = load_dataset(test_images_path, test_masks_path)


## 4. Définition du Modèle TernausNet
Nous définissons ici l'architecture du modèle TernausNet.

In [ ]:
# Définir le modèle TernausNet
def ternausnet(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encodeur
    c1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(p3)
    c4 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(c4)

    # Décodeur
    u5 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = tf.keras.layers.concatenate([u5, c3])
    c5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(u5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(c5)

    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c2])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(u6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c1])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(u7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(c7)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    return model


## 5. Compilation et Entraînement du Modèle
Nous allons maintenant compiler le modèle et définir les callbacks pour le suivi de la performance pendant l'entraînement.

In [ ]:
# Instancier le modèle
model = ternausnet((IMG_SIZE[0], IMG_SIZE[1], 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Définir les callbacks
checkpoint = ModelCheckpoint('datas/ternaunet_malaria_model.keras', save_best_only=True)

# Entraînement du modèle
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=BATCH_SIZE, callbacks=[checkpoint])


## 6. Sauvegarde du Modèle
Enfin, nous sauvegardons le modèle entraîné.

In [ ]:
# Sauvegarder le modèle
model.save('datas/ternausnet_malaria_model.keras')


## 7. Évaluation du Modèle
Cette section pourrait être utilisée pour évaluer les performances du modèle sur un jeu de données de test ou pour visualiser les prédictions.

In [ ]:
# Évaluation du modèle sur les données de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


## 8. Visualisation des Résultats
Vous pouvez inclure des visualisations des résultats, en affichant quelques images d'entrée avec leurs masques prédits.

In [ ]:
import matplotlib.pyplot as plt

# Visualisation des résultats
def visualize_results(images, true_masks, pred_masks, n=5):
    plt.figure(figsize=(20, 10))
    for i in range(n):
        plt.subplot(3, n, i + 1)
        plt.imshow(images[i])
        plt.title("Image")
        plt.axis("off")
        
        plt.subplot(3, n, i + 1 + n)
        plt.imshow(true_masks[i].squeeze(), cmap='gray')
        plt.title("Masque vrai")
        plt.axis("off")
        
        plt.subplot(3, n, i + 1 + 2 * n)
        plt.imshow(pred_masks[i].squeeze(), cmap='gray')
        plt.title("Masque prédit")
        plt.axis("off")
    plt.show()

# Prédictions
pred_masks = model.predict(X_test)

# Visualiser les résultats
visualize_results(X_test, y_test, pred_masks)
